In [35]:
%pip install xgboost
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
%pip install seaborn

Active code page: 1252
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1252
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1252
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1252
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1252
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
%matplotlib inline
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
import joblib
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error


In [37]:
data = pd.read_csv('valve_final.csv')

In [38]:
data.head(8)

,Date,Time,Player_count
0,24/10/2023,14:05,609952
1,24/10/2023,14:10,608506
2,24/10/2023,14:15,606577
3,24/10/2023,14:20,604879
4,24/10/2023,14:25,603868
5,24/10/2023,14:30,603070
6,24/10/2023,14:35,602284
7,24/10/2023,14:40,597245


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5930 entries, 0 to 5929
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          5930 non-null   object
 1   Time          5930 non-null   object
 2   Player_count  5930 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 139.1+ KB


In [40]:
data = data.drop(data[(data["Player_count"]== 0)].index)

In [41]:
data["New_date"] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5605 entries, 0 to 5929
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          5605 non-null   object        
 1   Time          5605 non-null   object        
 2   Player_count  5605 non-null   int64         
 3   New_date      5605 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 218.9+ KB


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5605 entries, 0 to 5929
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          5605 non-null   object        
 1   Time          5605 non-null   object        
 2   Player_count  5605 non-null   int64         
 3   New_date      5605 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 218.9+ KB


### Feature engineering (transform data to be used for the machine learning algo) 

In [44]:

data["New_time"] = pd.to_datetime(data['Time'], format='%H:%M')

In [45]:
data["year"] = data["New_date"].dt.year
data["month"] = data["New_date"].dt.month
data["day"] = data["New_date"].dt.day
data["hour"] = data["New_time"].dt.hour
data["min"] = data["New_time"].dt.minute

In [46]:
data.drop(columns=["Date","Time","New_date","New_time"], inplace=True)

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5605 entries, 0 to 5929
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Player_count  5605 non-null   int64
 1   year          5605 non-null   int32
 2   month         5605 non-null   int32
 3   day           5605 non-null   int32
 4   hour          5605 non-null   int32
 5   min           5605 non-null   int32
dtypes: int32(5), int64(1)
memory usage: 197.1 KB


In [48]:
data.head(2200)

,Player_count,year,month,day,hour,min
0,609952,2023,10,24,14,5
1,608506,2023,10,24,14,10
2,606577,2023,10,24,14,15
3,604879,2023,10,24,14,20
4,603868,2023,10,24,14,25
...,...,...,...,...,...,...
2241,856242,2023,11,1,7,50
2242,849313,2023,11,1,7,55
2243,837769,2023,11,1,8,0
2244,810234,2023,11,1,8,5


### Preapering for training 

### Models and testing

#### xgboost

In [49]:
tss = TimeSeriesSplit(n_splits=5, test_size=11, gap=11)
df = data.sort_index()


fold = 0
preds = []
scores = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    FEATURES = ['year', 'month', 'day', 'hour', 'min']
    TARGET = 'Player_count'

    X_train = train[FEATURES].values
    y_train = train[TARGET]

    X_test = test[FEATURES].values
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

[0]	validation_0-rmse:910061.17109	validation_1-rmse:756718.49801


C:\Users\Tommy\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [23:20:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[100]	validation_0-rmse:364577.91212	validation_1-rmse:301785.39964
[200]	validation_0-rmse:189956.67919	validation_1-rmse:123076.68917


[300]	validation_0-rmse:145070.73513	validation_1-rmse:69048.95777
[400]	validation_0-rmse:133503.87353	validation_1-rmse:49545.53285
[500]	validation_0-rmse:129591.71655	validation_1-rmse:40860.20754
[600]	validation_0-rmse:127179.60101	validation_1-rmse:37582.61062
[700]	validation_0-rmse:125661.25312	validation_1-rmse:32798.56175
[800]	validation_0-rmse:124481.81774	validation_1-rmse:28788.55197
[900]	validation_0-rmse:123549.44857	validation_1-rmse:26797.40631
[999]	validation_0-rmse:122823.99731	validation_1-rmse:22334.32832
[0]	validation_0-rmse:909709.90812	validation_1-rmse:809213.16913
[100]	validation_0-rmse:364423.81259	validation_1-rmse:311546.80566
[200]	validation_0-rmse:189695.76099	validation_1-rmse:131901.17804
[300]	validation_0-rmse:144878.12859	validation_1-rmse:76312.87773


C:\Users\Tommy\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [23:20:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


...

[400]	validation_0-rmse:133212.06801	validation_1-rmse:72793.74614
[500]	validation_0-rmse:129333.77679	validation_1-rmse:63442.95068
[600]	validation_0-rmse:126934.92779	validation_1-rmse:54473.52109
[700]	validation_0-rmse:125478.08810	validation_1-rmse:47574.86147
[800]	validation_0-rmse:124120.64352	validation_1-rmse:42584.22076
[900]	validation_0-rmse:123253.55435	validation_1-rmse:39993.52002
[999]	validation_0-rmse:122380.01349	validation_1-rmse:36437.01665
[0]	validation_0-rmse:909241.88465	validation_1-rmse:945542.47462
[100]	validation_0-rmse:364138.99676	validation_1-rmse:403929.94866
[200]	validation_0-rmse:189662.34137	validation_1-rmse:298356.03704
[300]	validation_0-rmse:144739.75795	validation_1-rmse:287939.75936


C:\Users\Tommy\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [23:20:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[400]	validation_0-rmse:133170.41461	validation_1-rmse:287221.56797
[444]	validation_0-rmse:131006.28765	validation_1-rmse:287720.65928
[0]	validation_0-rmse:909192.50544	validation_1-rmse:1106374.52747
[100]	validation_0-rmse:364046.70757	validation_1-rmse:455560.69858
[200]	validation_0-rmse:189691.33991	validation_1-rmse:239847.52994
[300]	validation_0-rmse:144580.75774	validation_1-rmse:160409.29497


C:\Users\Tommy\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [23:20:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[400]	validation_0-rmse:133095.09298	validation_1-rmse:124658.58636
[500]	validation_0-rmse:129244.40041	validation_1-rmse:102940.04288
[600]	validation_0-rmse:126804.11684	validation_1-rmse:89286.75491
[700]	validation_0-rmse:125318.86850	validation_1-rmse:81864.94457
[800]	validation_0-rmse:123922.25976	validation_1-rmse:81092.78468
[805]	validation_0-rmse:123848.13431	validation_1-rmse:80847.24710


In [50]:
print(f'Score across folds {np.mean(scores):0.4f}')
print(f'Fold scores:{scores}')

Score across folds 90122.1144
Fold scores:[22334.32836910176, 24881.414390905804, 36420.84514238101, 287218.50013671734, 79755.4841455396]


Saving model to pkl file for later use


In [51]:
reg.save_model("reg_model_valve.json")
